<a href="https://colab.research.google.com/github/pantheredeye/pantheredeye/blob/main/PointAddressMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install fuzzywuzzy

In [4]:
pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.3 MB/s eta 0:00:00


In [16]:
import pandas as pd
from fuzzywuzzy import fuzz

# Read the Excel worksheets or CSV files
occurrences = pd.read_excel('occurrences.xlsx', sheet_name='Sheet2', usecols=['Date', 'Address', 'Offense'])
point_addresses = pd.read_excel('point_addresses.xlsx', sheet_name='Sheet1', usecols=['OBJECTID', 'FULL_ADDR'])

# Create a dictionary for point addresses
point_addr_dict = dict(zip(point_addresses['FULL_ADDR'], point_addresses.index))

# Create an empty list to store the matches
matches = []

# Create a dictionary to store the matched addresses
matched_addresses = {}

# Loop through each occurrence address
for _, row in occurrences.iterrows():
    occ_addr = str(row['Address']).strip().upper()  # Convert to string, trim whitespace, and convert to uppercase
    date = row['Date']
    offense = row['Offense']

    # Check if the occurrence address has already been matched
    if occ_addr in matched_addresses:
        match, object_id, confidence = matched_addresses[occ_addr]
        matches.append((date, occ_addr, offense, match, object_id, confidence))
        continue

    # Step 1: Exact match
    if occ_addr in point_addr_dict:
        match = occ_addr
        object_id = point_addresses.loc[point_addr_dict[occ_addr], 'OBJECTID']
        confidence = 100
        matched_addresses[occ_addr] = (match, object_id, confidence)
        matches.append((date, occ_addr, offense, match, object_id, confidence))
        continue

    # Step 2: Number match and fuzzy matching
    occ_addr_parts = occ_addr.split(maxsplit=1)
    if len(occ_addr_parts) == 2:
        occ_addr_num, occ_addr_name = occ_addr_parts
        filtered_point_addresses = point_addresses[point_addresses['FULL_ADDR'].str.startswith(occ_addr_num)]

        if not filtered_point_addresses.empty:
            best_match = None
            best_confidence = 0

            for _, point_row in filtered_point_addresses.iterrows():
                point_addr_parts = point_row['FULL_ADDR'].split(maxsplit=1)
                if len(point_addr_parts) == 2:
                    _, point_addr_name = point_addr_parts
                    confidence = fuzz.ratio(occ_addr_name, point_addr_name)
                    if confidence > best_confidence:
                        best_match = point_row['FULL_ADDR']
                        best_confidence = confidence

            if best_match:
                object_id = point_addresses.loc[point_addresses['FULL_ADDR'] == best_match, 'OBJECTID'].values[0]
                matched_addresses[occ_addr] = (best_match, object_id, best_confidence)
                matches.append((date, occ_addr, offense, best_match, object_id, best_confidence))
                continue

    # Step 3: No match
    matched_addresses[occ_addr] = ('No Match', None, 0)
    matches.append((date, occ_addr, offense, 'No Match', None, 0))

# Create a dataframe from the matches list
matches_df = pd.DataFrame(matches, columns=['Date', 'OccurrenceAddress', 'Offense', 'MatchedAddress', 'OBJECTID', 'MatchConfidence'])

# Save the matches dataframe to a new Excel file or CSV
matches_df.to_excel('address_matches.xlsx', index=False)
# or
# matches_df.to_csv('address_matches.csv', index=False)